# Text generation using Keras and Tensorflow

From: https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/

## Load Data

In [52]:
from numpy import array
from pickle import dump
from pickle import load
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [32]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [33]:
# load text
raw_text = load_doc('sixpence.txt')
print(raw_text)

Sing a song of sixpence,
A pocket full of rye.
Four and twenty blackbirds,
Baked in a pie.
 
When the pie was opened
The birds began to sing;
Wasn't that a dainty dish,
To set before the king.
 
The king was in his counting house,
Counting out his money;
The queen was in the parlour,
Eating bread and honey.
 
The maid was in the garden,
Hanging out the clothes,
When down came a blackbird
And pecked off her nose.


In [34]:
# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

In [35]:
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))

Total Sequences: 1
Total Sequences: 2
Total Sequences: 3
Total Sequences: 4
Total Sequences: 5
Total Sequences: 6
Total Sequences: 7
Total Sequences: 8
Total Sequences: 9
Total Sequences: 10
Total Sequences: 11
Total Sequences: 12
Total Sequences: 13
Total Sequences: 14
Total Sequences: 15
Total Sequences: 16
Total Sequences: 17
Total Sequences: 18
Total Sequences: 19
Total Sequences: 20
Total Sequences: 21
Total Sequences: 22
Total Sequences: 23
Total Sequences: 24
Total Sequences: 25
Total Sequences: 26
Total Sequences: 27
Total Sequences: 28
Total Sequences: 29
Total Sequences: 30
Total Sequences: 31
Total Sequences: 32
Total Sequences: 33
Total Sequences: 34
Total Sequences: 35
Total Sequences: 36
Total Sequences: 37
Total Sequences: 38
Total Sequences: 39
Total Sequences: 40
Total Sequences: 41
Total Sequences: 42
Total Sequences: 43
Total Sequences: 44
Total Sequences: 45
Total Sequences: 46
Total Sequences: 47
Total Sequences: 48
Total Sequences: 49
Total Sequences: 50
Total Seq

In [36]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

## Train Language Model

### Load Data

In [37]:
# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

### Encode Sequences

In [38]:
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [39]:
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

In [40]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 38


In [41]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [42]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

## Fit Model

In [43]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                34200     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 1s - loss: 3.6239 - acc: 0.0877
Epoch 2/100
 - 0s - loss: 3.5396 - acc: 0.2030
Epoch 3/100
 - 0s - loss: 3.2678 - acc: 0.1905
Epoch 4/100
 - 0s - loss: 3.0874 - acc: 0.1905
Epoch 5/100
 - 0s - loss: 3.0165 - acc: 0.1905
Epoch 6/100
 - 0s - loss: 2.9863 - acc: 0.1905
Epoch 7/100
 - 0s - loss: 2.9686 - acc: 0.1905
Epoch 8/100
 - 0s - loss: 2.9514 - acc: 0.1905
Epoch 9/100
 - 0s - loss: 2.9319 - acc: 0.1930
Epoch 10/100
 - 0s - loss: 2.9014 - acc: 0.1955
Epoch 11/100
 - 0s - loss: 2.8799 - acc: 0.2080
Epoch 12/100
 - 0s - loss: 2.8546 - acc: 0.1980
Epoch 13/100
 - 0s - loss: 2.8145 - acc: 0.2180
Epoch 14/100
 - 0s - loss: 2.7856 - acc: 0.2180
Epoch 15/100
 - 0s - loss: 2.7589 - acc: 0.2231
Epoch 16/100
 - 0s - loss: 2.7284 - acc: 0.2406
Epoch 17/100
 - 0s - loss: 2.6592 - acc: 0.2556
Epoch 18/100
 - 0s - loss: 2.6222 - acc: 0.2531
Epoch 19/100
 - 0s - loss: 2.5697 - acc: 0.2957
Epoch 20/100
 - 0s - loss: 2.5240 - acc: 0.2832
E

## Save Model

In [45]:
# save the model to file
model.save('model.h5')

# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

## Generate Text

In [46]:
# load the model
model = load_model('model.h5')

# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

NameError: name 'load_model' is not defined

In [49]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

In [53]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))

Sing a song of sixpence, A poc
king was in his counting house
hello worl m aed a rleenyy ah 
